In [134]:
import pandas as pd
import requests
import json

In [130]:
# Loading in Olympic data
olympics_path = "athlete_events.csv"
noc_path = "noc_regions.csv"
census_path = "internationalcensus.csv"
gdp_path = "international_gdp.csv"

# Read the modified GoodReads csv and store into Pandas DataFrame
olympics_df = pd.read_csv(olympics_path)

noc_df = pd.read_csv(noc_path)

census = pd.read_csv(census)


In [153]:
# Loading in census data from World Bank
url = "http://api.worldbank.org/v2/country/all/indicator/"
api_format = "json"
population = "SP.POP.TOTL"
maxpages = "365"

country= []
year= []
pop =[]

for x in maxpages: 
    response = requests.get(f"{url}{population}?format={api_format}&page={x}").json()
    x = 1+x
    
print(f"{json.dumps(response, indent = 4)}")

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [149]:
# country_name, date, population
country_names, dates, populations = [], [], []
for item in response[1]:
    country_names.append(item["country"]["value"])
#     dates.append()
#     populations.append()
print(country_names)

['Africa Eastern and Southern', 'Africa Eastern and Southern', 'Africa Eastern and Southern', 'Africa Eastern and Southern', 'Africa Eastern and Southern', 'Africa Eastern and Southern', 'Africa Eastern and Southern', 'Africa Eastern and Southern', 'Africa Eastern and Southern', 'Africa Eastern and Southern', 'Africa Eastern and Southern', 'Africa Eastern and Southern', 'Africa Eastern and Southern', 'Africa Eastern and Southern', 'Africa Eastern and Southern', 'Africa Eastern and Southern', 'Africa Eastern and Southern', 'Africa Eastern and Southern', 'Africa Eastern and Southern', 'Africa Eastern and Southern', 'Africa Eastern and Southern', 'Africa Eastern and Southern', 'Africa Eastern and Southern', 'Africa Eastern and Southern', 'Africa Eastern and Southern', 'Africa Eastern and Southern', 'Africa Eastern and Southern', 'Africa Eastern and Southern', 'Africa Eastern and Southern', 'Africa Eastern and Southern', 'Africa Eastern and Southern', 'Africa Eastern and Southern', 'Africa

In [95]:
merge_df = pd.merge(olympics_df, noc_df, on="NOC")

olympiccountries_df = merge_df[["region","Year","Season","City","Sport","Event","Medal"]]
olympiccountries_df

,region,Year,Season,City,Sport,Event,Medal
0,China,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,China,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,China,2000,Summer,Sydney,Boxing,Boxing Men's Middleweight,NaN
3,China,2004,Summer,Athina,Wrestling,"Wrestling Men's Lightweight, Greco-Roman",NaN
4,China,2016,Summer,Rio de Janeiro,Swimming,Swimming Women's 200 metres Freestyle,NaN
...,...,...,...,...,...,...,...
270762,Lesotho,2008,Summer,Beijing,Athletics,Athletics Women's Marathon,NaN
270763,Lesotho,2012,Summer,London,Athletics,Athletics Women's Marathon,NaN
270764,Lesotho,1996,Summer,Atlanta,Athletics,Athletics Women's 4 x 100 metres Relay,NaN
270765,Lesotho,1984,Summer,Los Angeles,Boxing,Boxing Men's Welterweight,NaN


In [41]:
goldcountries_df = olympiccountries_df.loc[olympiccountries_df["Medal"] == "Gold", :]
silvercountries_df = olympiccountries_df.loc[olympiccountries_df["Medal"] == "Silver", :]
bronzecountries_df = olympiccountries_df.loc[olympiccountries_df["Medal"] == "Bronze", :]

countries = olympiccountries_df["Team"]
gold= []
silver= []
bronze= []

for country in countries:
    gold.append(goldcountries_df["Team"].nunique())
    silver.append(silvercountries_df["Team"].nunique())
    bronze.append(bronzecountries_df["Team"].nunique())
    
countrymedals_df = pd.DataFrame({"Countries": countries, "Gold": gold, "Silver": silver, "Bronze": bronze})

countrymedals_df



KeyboardInterrupt: 

In [103]:
countrymedal_group = olympiccountries_df.groupby(["region"])
medal_counts = countrymedal_group["Medal"].value_counts()
countrymedal_df = pd.DataFrame(medal_counts)
countrymedal_df = countrymedal_df.rename(columns={'region':'Country','Medal': 'Medal','Medal':'Total Medals'})
countrymedal_df

Total Medals
region      Medal               
Afghanistan Bronze             2
Algeria     Bronze             8
            Gold               5
            Silver             4
Argentina   Silver            92
...                          ...
Zambia      Bronze             1
            Silver             1
Zimbabwe    Gold              17
            Silver             4
            Bronze             1

[327 rows x 1 columns]

In [121]:
total_athletes = countrymedal_group["Sport"].value_counts()
topsportbycountry_df= pd.DataFrame(total_athletes)
topsportbycountry_df = topsportbycountry_df.rename(columns={'region':'Country','Sport': 'Sport','Sport':'Athletes'})
topsportbycountry_df = topsportbycountry_df.sort_values(["Athletes"], ascending=False)
topsportbycountry_df

,,Athletes
region,Sport,
USA,Athletics,3211
UK,Athletics,2244
Germany,Athletics,2020
USA,Gymnastics,2013
Germany,Gymnastics,1856
...,...,...
Grenada,Taekwondo,1
Philippines,Luge,1
Guam,Biathlon,1


In [119]:
totalmedals = countrymedal_group["Medal"].count()
totalmedals_df = pd.DataFrame(totalmedals)
totalmedals_df = totalmedals_df.rename(columns={'region':'Country','Medal':'Total Medals'})
totalmedals_df = totalmedals_df.sort_values(["Total Medals"], ascending=False)
totalmedals_df

,Total Medals
region,
USA,5637
Russia,3947
Germany,3756
UK,2068
France,1777
...,...
Equatorial Guinea,0
Gambia,0
Guam,0


In [125]:
summervswinter = countrymedal_group["Season"].value_counts()
season_df = pd.DataFrame(summervswinter)
season_df = season_df.rename(columns={'region':'Country','Season':'Season','Season':'Athletes'})
season_df

Athletes
region             Season          
Afghanistan        Summer       126
Albania            Summer        63
                   Winter         7
Algeria            Summer       539
                   Winter        12
...                             ...
Virgin Islands, US Winter        55
Yemen              Summer        48
Zambia             Summer       183
Zimbabwe           Summer       321
                   Winter         2

[316 rows x 1 columns]